<a href="https://colab.research.google.com/github/keshav434/Sentiment-Analysis/blob/main/Sentimental_Part_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
import re
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from math import log
import nltk
nltk.download('punkt')

# Step 1: Data Preprocessing
mydata = pd.read_csv('/content/drive/My Drive/deforestation_sentiment_train.csv')

def clean_text(sentence):

  #Removes all special characters from sentence. It will also strip out
  #extra whitespace and makes the string lowercase.'''

   return re.sub(r'[\\\\/:*«`\'?¿";!<>,.|]', '', sentence.lower().strip())


mydata['clean_text'] = mydata['text'].apply(clean_text)
print("2. Clean Text -----------")
print(mydata['clean_text'])
print(mydata.columns)
print(mydata.head(5))

# Step 2: Building the Vocabulary
def create_vocabulary(texts):
    vocabulary = set()
    for text in texts:
        words = word_tokenize(text)
        vocabulary.update(words)
    return vocabulary

vocabulary = create_vocabulary(mydata['clean_text'])
print("Vacabulary......")
print(vocabulary)
print("End of Vocabulary....")
# Step 3: Calculating Probabilities
X = mydata["clean_text"]
y = mydata["sentiments_list"]  # sentiment label
# y = ['', "{'negative'}", "{'neutral', 'negative'}" .. ]
total_samples = len(y)
print("total_samples->",total_samples)
print("Converting sentiments_list to Dict")
dict_sentiments=dict(mydata['sentiments_list'].value_counts())
print(dict_sentiments)
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
neg_count = dict_sentiments["{'negative'}"]
pos_count = dict_sentiments["{'positive'}"]


prior_negative = neg_count /total_samples
prior_positive = pos_count / total_samples
print("Positive Count = ", pos_count)
print("Negative Count = ", neg_count)
print("prior_positive score =", prior_positive, "\nprior_negative score =", prior_negative)

def calculate_c_probabilities(texts, sentiments_list, sentiment, vocabulary):
    word_counts = defaultdict(int)
    total_sentiments = len(sentiments_list)
    for i in range(total_sentiments):
        if sentiments_list[i] == sentiment:
            words = word_tokenize(texts[i])
            for word in words:
                if word in vocabulary:
                    word_counts[word] += 1
    total_words = sum(word_counts.values())
    c_probabilities = {}
    for word in vocabulary:
        c_probabilities[word] = (word_counts[word] + 1) / (total_words + len(vocabulary))
    return c_probabilities

positive_c_prob = calculate_c_probabilities(mydata['clean_text'], mydata['sentiments_list'], "{'positive'}", vocabulary)
negative_c_prob = calculate_c_probabilities(mydata['clean_text'], mydata['sentiments_list'], "{'negative'}", vocabulary)

# Step 4: Sentiment Classification
def classify_sentiment(text):
    words = word_tokenize(text)
    positive_prob = log(prior_positive)
    negative_prob = log(prior_negative)
    for word in words:
        if word in vocabulary:
            positive_prob += log(positive_c_prob[word])
            negative_prob += log(negative_c_prob[word])
    return "positive" if positive_prob > negative_prob else "negative"

# Testing the classifier
test_text = "Our environment is good"
classified_sentiment = classify_sentiment(test_text)
print("Test Text:", test_text)
print("Predicted Sentiment:", classified_sentiment)


Mounted at /content/drive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


2. Clean Text -----------
0      three palm oil firms were responsible for a qu...
1      despicable #brazil govt using #pandemic as cov...
2      🌳🍂🌳impacto humano no #desmatamento  \nhuman im...
3      deforestation will be the end of the human rac...
4      @_wrong_guy_ @zerohedge no other codes nowdays...
                             ...                        
934    @greenworld_uk @thegreenparty sir david attenb...
935    the silvery gibbon is of genus hylobates which...
936      how to stop deforestation 🌲💪 httpstcoa9r2zcmeto
937    do you accept the challenge before entering wh...
938    the more lighter we take the subject of defore...
Name: clean_text, Length: 939, dtype: object
Index(['Unnamed: 0', 'id', 'text', 'user_name', 'user_description',
       'user_location', 'user_following', 'user_followers', 'tweets_by_user',
       'user_created_at', 'tweet_created', 'retweets', 'likes', 'hashtags',
       'media', 'search_term', 'emojis', 'sentiments_list',
       'revised_sent